In [2]:
import cv2
print(cv2.__version__)
print(dir(cv2.face))


4.11.0
['BIF', 'BIF_create', 'BasicFaceRecognizer', 'EigenFaceRecognizer', 'EigenFaceRecognizer_create', 'FaceRecognizer', 'Facemark', 'FacemarkAAM', 'FacemarkKazemi', 'FacemarkLBF', 'FacemarkTrain', 'FisherFaceRecognizer', 'FisherFaceRecognizer_create', 'LBPHFaceRecognizer', 'LBPHFaceRecognizer_create', 'MACE', 'MACE_create', 'MACE_load', 'PredictCollector', 'StandardCollector', 'StandardCollector_create', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_native', 'createFacemarkAAM', 'createFacemarkKazemi', 'createFacemarkLBF', 'drawFacemarks', 'getFacesHAAR', 'loadDatasetList', 'loadFacePoints', 'loadTrainingData']


In [3]:
import cv2

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if faces is None or len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face  # Return only the first face found
    
    cap = cv2.VideoCapture(1)
    user_id = 1
    img_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        cropped = face_cropped(frame)
        if cropped is not None:
            img_id += 1
            face = cv2.resize(cropped, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"data/user.{user_id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)
        
        if cv2.waitKey(1) == 13 or img_id == 200:  # Enter key to stop
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

generate_dataset()


In [4]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy
 
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

In [1]:
import cv2
import numpy as np
import os

# Function to draw face recognition boundary
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf):
    try:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except Exception as e:
        print(f"[ERROR] cvtColor failed: {e}")
        return img

    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)

        try:
            id, pred = clf.predict(gray_img[y:y + h, x:x + w])
            confidence = int(100 * (1 - pred / 300))
        except Exception as e:
            print(f"[ERROR] Prediction failed: {e}")
            continue

        if confidence > 70:
            name = "Unknown"
            if id == 1:
                name = "Richa"
            elif id == 2:
                name = "Devanshi"
            cv2.putText(img, f"{name} ({confidence}%)", (x, y - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

    return img


# Load Haar Cascade
cascade_path = "haarcascade_frontalface_default.xml"
if not os.path.exists(cascade_path):
    raise FileNotFoundError(f"[ERROR] Haar cascade file not found: {cascade_path}")

faceCascade = cv2.CascadeClassifier(cascade_path)

# Load trained model
model_path = "classifier.xml"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"[ERROR] Trained model file not found: {model_path}")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read(model_path)

# Open video capture (try 0 or 1 depending on your system)
video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    raise RuntimeError("[ERROR] Could not open webcam. Try changing index to 1 or 2.")

print("[INFO] Starting real-time face recognition. Press 'Enter' to exit.")

# Real-time recognition loop
while True:
    ret, img = video_capture.read()
    if not ret:
        print("[WARNING] Frame not captured from webcam.")
        continue

    img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), clf)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:  # Press Enter to exit
        print("[INFO] Exiting...")
        break

video_capture.release()
cv2.destroyAllWindows()


[INFO] Starting real-time face recognition. Press 'Enter' to exit.
[INFO] Exiting...
